In [ ]:
# !pip install langchain
# !pip install -U langchain-openai
# !pip install -U langchain_neo4j
# !pip install langchain_community
# !pip install neo4j
# !pip install shapely
# !pip install -U pyproj
# !pip install tqdm
# !pip install langchain-community
# !pip install -U pandas

# !pip install -U langchain-neo4j

In [15]:
# import json
# from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv()

import os

from langchain_neo4j import Neo4jGraph
from langchain.chat_models import init_chat_model
from langgraph.graph import START, StateGraph
from langchain_core.prompts import PromptTemplate
from typing_extensions import List, TypedDict


import psycopg2
from neo4j import GraphDatabase
from decimal import Decimal
from shapely import wkb
from pyproj import CRS, Transformer


In [16]:

load_dotenv()

keys = ["NEO4J_USER", "NEO4J_PASSWORD","NEO4J_URI_LOCAL", "NEO4J_USER_LOCAL", "NEO4J_PASSWORD_LOCAL"]
for key in keys:
    if key not in os.environ:
        raise Exception(f"Key '{key}' not found not in .env")
    
print("Credentials from .env file successfully loaded")

NEO4J_LOCAL_DATABASE = "busoppdse203"

Credentials from .env file successfully loaded


In [17]:
graph = Neo4jGraph(url="bolt://localhost:7687",    username='neo4j',password='admin123',database=NEO4J_LOCAL_DATABASE )
graph.query("MATCH (n) RETURN n LIMIT 1;")


[{'n': {'code': 'CA', 'name': 'Calfironia', 'id': 1}}]

In [19]:
# Initialize the LLM
model_ver = "gpt-4o"
model_provider = 'openai'
model = init_chat_model(model_ver, model_provider=model_provider)


In [20]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[dict]
    answer: str


In [21]:
# Retrieve context 
def retrieve(state: State):
    context = graph.query("CALL db.schema.visualization()")
    return {"context": context}


In [22]:
# Create a prompt
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

Answer:"""

prompt = PromptTemplate.from_template(template)



In [23]:
# Generate the answer based on the question and context
def generate(state: State):
    messages = prompt.invoke({"question": state["question"], "context": state["context"]})
    response = model.invoke(messages)
    return {"answer": response.content}

# Define application steps
workflow = StateGraph(State).add_sequence([retrieve, generate])
workflow.add_edge(START, "retrieve")
app = workflow.compile()


question = "How is the graph structured?"
# question = "Are any cities contained in cities"

response = app.invoke({"question": question})
print("Answer:", response["answer"])



Answer: The graph is structured with nodes and relationships. The nodes represent different geographic and business entities, such as "SpatialLayer," "BusinessLocation," "State," "Zipcode," "City," "County," "Community," "Business," and "BlockGroup." The relationships describe how these entities are connected to each other. For example:

1. "Community" can be "contained_in" both "City" and "County."
2. "City" can be "contained_in" another "City" or "County."
3. Entities can be "nearby" each other, such as "City" and "Community."
4. Entities can be "adjacent_to" each other, including "City," "Community," and "County."

The graph can represent various types of spatial and administrative relationships within these geographical and business contexts.
